# Music Genre Prediction

In [1]:
class Config:
    NB = '207'
    dataset_NB = '103'

    raw_data_dir = '../data/raw/'
    processed_data_dir = '../data/processed/'
    interim_dir = '../data/interim/'
    submission_dir = '../data/submission/'

    random_seed = 42
    n_folds = 5

    row_id = 'index'
    target = 'genre'

## Import libralies

In [2]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')

In [28]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

plotly_template = dict(
    layout=go.Layout(
        template='plotly_dark',
        font=dict(
            family="Franklin Gothic",
            size=12
        ),
        height=500,
        width=1000,
    )
)


color_palette = {
    'Bin': ['#016CC9','#E876A3'],
    'Cat5': ['#E876A3', '#E0A224', '#63B70D', '#6BCFF6', '#13399E'],
}

In [17]:
import random
import joblib
import itertools
from itertools import combinations
from imblearn import FunctionSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import StratifiedKFold, GroupKFold, train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import roc_auc_score, roc_curve, auc, f1_score, confusion_matrix
import scipy.stats as stats
import time

import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import optimizers
from keras.utils import np_utils

In [5]:
df_train = pd.read_pickle(Config.processed_data_dir + f'nb{Config.dataset_NB}_train.pkl', compression='zip')
df_test = pd.read_pickle(Config.processed_data_dir + f'nb{Config.dataset_NB}_test.pkl', compression='zip')

submission = pd.read_csv(Config.raw_data_dir + 'sample_submit.csv', header=None)

df_train.shape

(4046, 285)

In [33]:
df_train.describe()

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo_int,region_A,region_B,region_C,region_D,region_E,region_F,region_G,region_H,region_I,region_J,region_K,region_L,region_M,region_N,region_O,region_P,region_Q,region_R,region_S,region_T,unknown,duration_long,popularity_add_duration_ms,popularity_sub_duration_ms,popularity_mul_duration_ms,popularity_bigger_duration_ms,popularity_add_acousticness,popularity_sub_acousticness,popularity_mul_acousticness,popularity_bigger_acousticness,popularity_add_positiveness,popularity_sub_positiveness,popularity_mul_positiveness,popularity_bigger_positiveness,popularity_add_danceability,popularity_sub_danceability,popularity_mul_danceability,popularity_bigger_danceability,popularity_add_loudness,popularity_sub_loudness,popularity_mul_loudness,popularity_bigger_loudness,popularity_add_energy,popularity_sub_energy,popularity_mul_energy,popularity_bigger_energy,popularity_add_liveness,popularity_sub_liveness,popularity_mul_liveness,popularity_bigger_liveness,popularity_add_speechiness,popularity_sub_speechiness,popularity_mul_speechiness,popularity_bigger_speechiness,popularity_add_instrumentalness,popularity_sub_instrumentalness,popularity_mul_instrumentalness,popularity_bigger_instrumentalness,popularity_add_tempo_int,popularity_sub_tempo_int,popularity_mul_tempo_int,popularity_bigger_tempo_int,duration_ms_add_acousticness,duration_ms_sub_acousticness,duration_ms_mul_acousticness,duration_ms_bigger_acousticness,duration_ms_add_positiveness,duration_ms_sub_positiveness,duration_ms_mul_positiveness,duration_ms_bigger_positiveness,duration_ms_add_danceability,duration_ms_sub_danceability,duration_ms_mul_danceability,duration_ms_bigger_danceability,duration_ms_add_loudness,duration_ms_sub_loudness,duration_ms_mul_loudness,duration_ms_bigger_loudness,duration_ms_add_energy,duration_ms_sub_energy,duration_ms_mul_energy,duration_ms_bigger_energy,duration_ms_add_liveness,duration_ms_sub_liveness,duration_ms_mul_liveness,duration_ms_bigger_liveness,duration_ms_add_speechiness,duration_ms_sub_speechiness,duration_ms_mul_speechiness,duration_ms_bigger_speechiness,duration_ms_add_instrumentalness,duration_ms_sub_instrumentalness,duration_ms_mul_instrumentalness,duration_ms_bigger_instrumentalness,duration_ms_add_tempo_int,duration_ms_sub_tempo_int,duration_ms_mul_tempo_int,duration_ms_bigger_tempo_int,acousticness_add_positiveness,acousticness_sub_positiveness,acousticness_mul_positiveness,acousticness_bigger_positiveness,acousticness_add_danceability,acousticness_sub_danceability,acousticness_mul_danceability,acousticness_bigger_danceability,acousticness_add_loudness,acousticness_sub_loudness,acousticness_mul_loudness,acousticness_bigger_loudness,acousticness_add_energy,acousticness_sub_energy,acousticness_mul_energy,acousticness_bigger_energy,acousticness_add_liveness,acousticness_sub_liveness,acousticness_mul_liveness,acousticness_bigger_liveness,acousticness_add_speechiness,acousticness_sub_speechiness,acousticness_mul_speechiness,acousticness_bigger_speechiness,acousticness_add_instrumentalness,acousticness_sub_instrumentalness,acousticness_mul_instrumentalness,acousticness_bigger_instrumentalness,acousticness_add_tempo_int,acousticness_sub_tempo_int,acousticness_mul_tempo_int,acousticness_bigger_tempo_int,positiveness_add_danceability,positiveness_sub_danceability,positiveness_mul_danceability,positiveness_bigger_danceability,positiveness_add_loudness,positiveness_sub_loudness,positiveness_mul_loudness,positiveness_bigger_loudness,positiveness_add_energy,positiveness_sub_energy,positiveness_mul_energy,positiveness_bigger_energy,positiveness_add_liveness,positiveness_sub_liveness,positiveness_mul_liveness,positiveness_bigger_liveness,positiveness_add_speechiness,positiveness_sub_speechiness,positiveness_mul_speechiness,positiveness_bigger_speechiness,positiveness_add_instrumentalness,positiveness_sub_instrumentalness,pos

## Parameter Setting

In [6]:
features = [col for col in df_train.columns if col not in [Config.row_id, Config.target]]

In [ ]:
# bool値をintに変換
col_list = [col for col in df_train.columns if df_train[col].dtypes == bool]

for df in [df_train, df_test]:
    df[col_list] = df[col_list] * 1

df_train.dtypes

## Validation data Setting

In [7]:
X_test = df_test[features]

'''
for c in TARGET_ENCODING_CATEGORY:
    data_tmp = pd.DataFrame({c: df_train[c], 'target': df_train[TARGET]})
    target_mean = data_tmp.groupby(c)['target'].mean()
    X_test.loc[:, c] = X_test[c].map(target_mean)
'''

X_test = (X_test.values).astype(np.float32)
X_test.shape

(4046, 283)

## Modeling

### Multi Layer Perceptron
- 隠れ層3層のMLP
- kernel_initializerにHeの初期化を採用
- Batch Normalizationを採用
- 活性化関数にReLUを採用
- Optimizerを採用（SGD、Adamなど。）
- Dropoutを採用
  - DropoutとBatchNormalizationを同時に使うと学習がうまくできない場合がある。
  - その場合、Dropoutを外す
- モデルの順序は、BatchNormalization、活性化関数、Dropoutであることに注意

In [31]:
def root_mean_squared_error(y_true, y_pred):
    return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))

def setup_model():
    activation = 'relu'
    kernel_initializer = 'he_normal'

    model = Sequential()

    model.add(Dense(256, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(0.25))

    model.add(Dense(192, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(0.25))

    model.add(Dense(128, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    # model.add(Dropout(0.25))

    model.add(Dense(64, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    # model.add(Dropout(0.25))

    model.add(Dense(32, kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    # model.add(Dropout(0.25))

    model.add(Dense(11, activation='softmax'))

    optimizer = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=True)
    # optimizer = optimizers.SGD(learning_rate=0.001)

    # model.compile(optimizer=optimizer, loss=root_mean_squared_error, metrics=[root_mean_squared_error])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_crossentropy'])

    return model


def setup_callbacks():
    es = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
    lr = ReduceLROnPlateau(monitor="val_loss", factor=0.7, patience=5, verbose=1)
    callbacks = [es, lr]

    return callbacks


mlp_param = {
    'epochs': 300,
    'batch_size': 100,
    'verbose': 1,
}


### Training & Validation with TargetEncoding

In [32]:
np.random.seed(Config.random_seed)
tf.random.set_seed(Config.random_seed)

# Create a numpy array to store test predictions
test_predictions = np.zeros((len(df_test), Config.n_folds))

# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(df_train))

feature_importance_df = pd.DataFrame(index=features)
y_valids, val_preds =[],[]

kfold = StratifiedKFold(n_splits=Config.n_folds, shuffle=True, random_state=Config.random_seed)

for fold, (train_idx, valid_idx) in enumerate(kfold.split(df_train, df_train[Config.target])):

    print(' ')
    print('-'*50)
    print(f'Training fold {fold+1} with {len(features)} features...')

    X_train, X_val = df_train[features].iloc[train_idx], df_train[features].iloc[valid_idx]
    y_train, y_val = df_train[Config.target].iloc[train_idx], df_train[Config.target].iloc[valid_idx]

    # Over Sampling
    sm = SMOTE(random_state=Config.random_seed)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    # print(y_train.value_counts())

    dummy_y_train = np_utils.to_categorical(y_train)
    dummy_y_val = np_utils.to_categorical(y_val)

    # training
    model = setup_model()
    callbacks = setup_callbacks()
    hist = model.fit(X_train, dummy_y_train, validation_data=(X_val, dummy_y_val), epochs=mlp_param['epochs'], batch_size=mlp_param['batch_size'], callbacks=callbacks, verbose=mlp_param['verbose'])

    print(f'================================== training {fold+1} fin. ==================================')

    # Predict validation data
    print(f'================================== validation-data predicting ... ==================================')
    val_pred = model.predict(X_val)
    val_pred = np.argmax(val_pred, axis=1)
    oof_predictions[valid_idx] = val_pred

    # Predict test data
    print(f'================================== test-data predicting ... ==================================')
    test_pred = model.predict(df_test[features])
    test_pred = np.argmax(test_pred, axis=1)

    test_predictions[:, fold] += test_pred

    # save results
    y_valids.append(y_val)
    val_preds.append(val_pred)
    # feature_importance_df["Importance_Fold"+str(fold+1)]=model.feature_importance(importance_type='gain')

    # Compute fold metric
    val_pred = pd.DataFrame(data={'prediction': val_pred})
    y_val = pd.DataFrame(data={'target': y_val.reset_index(drop=True)})
    score = f1_score(y_val, val_pred, average='macro')

    print(f'Fold {fold+1} CV result')
    print(f'metric : {score}')

    del X_train, X_val, y_train, y_val
    _ = gc.collect()

# Compute out of folds metric
oof_predictions = pd.DataFrame(data={'prediction': oof_predictions})
y_true = pd.DataFrame(data={Config.target: df_train[Config.target]})

print(' ')
print('-'*50)
print(f'TOTAL socre : {f1_score(df_train[Config.target], oof_predictions["prediction"], average="macro")}')
print('-'*50)

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame({Config.row_id: df_train[Config.row_id], Config.target: df_train[Config.target], 'prediction': oof_predictions['prediction']})

# Create a dataframe to store test prediction
test_predictions, _ = stats.mode(test_predictions, axis=1)
test_predictions = test_predictions.reshape(-1)

test_df = pd.DataFrame({Config.row_id: df_test[Config.row_id], Config.target: test_predictions})

 
--------------------------------------------------
Training fold 1 with 283 features...
Epoch 1/300
118/118 [==============================] - 2s 8ms/step - loss: 1.5692 - categorical_crossentropy: 1.5692 - val_loss: 2.9320 - val_categorical_crossentropy: 2.9320 - lr: 0.0100
Epoch 2/300
118/118 [==============================] - 1s 5ms/step - loss: 1.2697 - categorical_crossentropy: 1.2697 - val_loss: 1.8301 - val_categorical_crossentropy: 1.8301 - lr: 0.0100
Epoch 3/300
118/118 [==============================] - 1s 5ms/step - loss: 1.1144 - categorical_crossentropy: 1.1144 - val_loss: 1.8781 - val_categorical_crossentropy: 1.8781 - lr: 0.0100
Epoch 4/300
118/118 [==============================] - 1s 5ms/step - loss: 1.0144 - categorical_crossentropy: 1.0144 - val_loss: 1.7091 - val_categorical_crossentropy: 1.7091 - lr: 0.0100
Epoch 5/300
118/118 [==============================] - 1s 5ms/step - loss: 0.9694 - categorical_crossentropy: 0.9694 - val_loss: 1.8598 - val_categorical_cros

KeyboardInterrupt: 

In [ ]:
# Save results
oof_df_tmp = oof_df.drop(columns=[Config.target])
oof_df_tmp.columns = [Config.row_id, f'nb{Config.NB}']
oof_df_tmp.to_csv(Config.interim_dir + f'nb{Config.NB}.csv', index=False)
oof_df_tmp

## 結果の可視化

In [29]:
cm = confusion_matrix(oof_df[Config.target], oof_df['prediction'], normalize='true')

names = [f'Target_{i}' for i in range(11)]

fig = ff.create_annotated_heatmap(cm, x=names, y=names)
fig.update_layout(
    yaxis_title='True Label',
    xaxis_title='Pred Label',
)
fig.show()

In [30]:
fig = go.Figure(layout=plotly_template['layout'])
fig.add_trace(
    go.Histogram(
        x=test_df[Config.target],
        name=f'Prediction',
        histnorm='probability',
        marker=dict(color=color_palette['Bin'][0]),
        #line=dict(color='black')
    ),
)

fig.add_trace(
    go.Histogram(
        x=df_train[Config.target],
        name=f'Train',
        histnorm='probability',
        marker=dict(color=color_palette['Bin'][1]),
        opacity=0.5
        #line=dict(color='black')
    ),
)

fig.update_layout(
    title='Prediction Distribution',
    barmode='overlay',
    uniformtext_minsize=15,
    uniformtext_mode='hide',
    width=700)

fig.show()

## Submission

In [ ]:
Config.NB

In [ ]:
test_df.to_csv(Config.submission_dir + f'nb{Config.NB}.csv', index=False)

## 検証メモ

In [ ]:
df_train[features].dtypes